In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Got ya boi

## To be honest...
I actually tried to be smart and do things like handcraft feature engineering and data scaling and stuff like that, but in the end it was just AutoML, so...


- Feature engineering model val roc_auc result: 0.8688
- No feature engineering val roc_auc: 0.89

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
class cfg:
    train_path = '/kaggle/input/playground-series-s4e7/train.csv'
    test_path = '/kaggle/input/playground-series-s4e7/test.csv'
    sub_path = '/kaggle/input/playground-series-s4e7/sample_submission.csv'


# Load Dataset

In [ ]:
df  = pd.read_csv(cfg.train_path, index_col=0)
df

# Inspecting and Cleansing Data 

After the inspection, there is no need for na drop so we good to go.

In [ ]:
df.info()

inspecting nan value

In [ ]:
df.isna().sum()

# Data transformation

## Categorical data
- Gender
- Vehicle_Age
- Vehicle_Damage


## numerical data
- Age
- Driving_License
- Region_Code
- Annual_Premium
- Policy_Sales_Channel
- Vintage
- Response (leave them alone!) 

## Inspecting

### Gender

In [ ]:
df['Gender'].unique()

### Vehicle Age
In this case the value have meaning in orderly way so we can use **Ordinal Encoding** performance wise this make more sense than one-hot encoder. 

In [ ]:
df['Vehicle_Age'].unique()

### Vehicle Damage

In [ ]:
df['Vehicle_Damage'].unique()

# Data Analysis

Plot feature correlation

In [ ]:
df_tT = df.copy()

# calculate the correlation matrix on the numeric columns
corr = df_tT.select_dtypes('number').corr()

# plot the heatmap
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(corr, cmap="Blues", annot=True, ax=ax)

In [ ]:
df_tT

In [ ]:
df_tT.info()

# Feature Engineer

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# Normalization
def engineer_features(df):
    # Make a copy of the dataframe
    df_new = df.copy()
    
    # Categorical Encoding
    df_new['Gender'] = df_new['Gender'].map({'Female': 0, 'Male': 1})
    df_new['Vehicle_Damage'] = df_new['Vehicle_Damage'].map({'No': 0, 'Yes': 1})
    
    # Ordinal Encoding
    df_new['Vehicle_Age'] = df_new['Vehicle_Age'].map({'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2})
        
    # Interaction Features
    df_new['Age_Vehicle_Interaction'] = (df_new['Age'] * (df_new['Vehicle_Age']+1))
    df_new['Gender_VehicleDamage_Interaction'] = (df_new['Gender']+1) * (df_new['Vehicle_Damage']+1)**2
    df_new['Annual_Premium_Vintage_Interaction'] = (df_new['Annual_Premium']) * (df_new['Vintage']+1)
    
    df_new = df_new.drop(columns=['Driving_License', 'Vintage', 'Annual_Premium'])

    return df_new

# Apply feature engineering
df_transformed = engineer_features(df_tT)

In [ ]:
df_transformed

In [ ]:
# calculate the correlation matrix on the numeric columns
corr = df_transformed.select_dtypes('number').corr()

# plot the heatmap
fig, ax = plt.subplots(figsize=(10,10)) 
sns.heatmap(corr, cmap="Blues", annot=True, ax=ax)

In [ ]:
corr['Response']

In [ ]:
# ! pip install shapely>=2.0.1
# ! pip install autogluon.tabular[all] 
# ! pip show autogluon

In [ ]:
test_df = pd.read_csv(cfg.test_path, index_col=0)
test_df = engineer_features(test_df)
test_df 

In [ ]:
sub_df = pd.read_csv(cfg.sub_path)
sub_df

In [ ]:
from autogluon.tabular import TabularPredictor, TabularDataset
hyperparameters = {
    "XGB": {
        "colsample_bytree": 0.516652313273348,
        "enable_categorical": True,
        "learning_rate": 0.015,
        "max_depth": 9,
        "min_child_weight": 0.8567068904025429,
        "max_bin": 2**18-1,
        "n_estimators": 150000,
    },
}


In [ ]:
predictor = TabularPredictor(
    label='Response',
    eval_metric='roc_auc',
).fit(
    train_data=df,
    hyperparameters=hyperparameters,
    time_limit=3600,  # out of necessity, would increase to as high as possible if time permitted
    verbosity=4,  # to log val score every iteration of XGBoost,
#      num_gpus=1,
)

In [ ]:
predictions =  predictor.predict_proba(test_df, as_multiclass=False)

In [ ]:
sample = predictions.reset_index()
sample

In [ ]:
sample.to_csv('submission_new_2.csv', index=False)